In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict

In [ ]:
rate_to_use = ['stre', 'spd', 'jmp', 'endu', 'ins', 'dnk', 'ft', 'fg', 'tp', 'oiq', 'diq', 'drb', 'pss', 'reb', 'hgt']

In [ ]:
sim = json.load(open('newresign.json','rb'))
data = json.load(open('real-player.json','rb'))

In [ ]:
picks = defaultdict(list)
i =0 
for p in sim['players']:
    if int(p['draft']['year']) < 2021:
        continue
    pick = (p['draft']['round']-1)*30 + p['draft']['pick']
    vec = [p['ratings'][0][k] for k in rate_to_use]
    vec = np.array(vec)
    picks[pick].append(vec)
    i+=1

In [ ]:
picks_norm = {k: np.array(v) for k,v in picks.items() if len(v) > 1}
picks_norm = {k: v/v.sum(1,keepdims=True) for k,v in picks_norm.items()}

In [ ]:
roster = json.load(open('1984_draft.json','rb'))

In [ ]:
for p in roster['players']:
    if 'srID' not in p:
        continue
    srId = p['srID']
    if srId not in data['bios']:
        continue
    bio = data['bios'][srId]
    pick = (bio['draftRound']-1)*30 + bio['draftPick']
    vec = np.array([p['ratings'][0][k] for k in rate_to_use])
    vecN = vec/vec.sum()
    if pick not in picks_norm:
        pick = -30
    diff = np.linalg.norm(picks_norm[pick]- vecN,axis=1)
    diff_idx = np.argmin(diff)
    stats = picks[pick][diff_idx]
    del p['ratings'][0]['ovr']
    del p['ratings'][0]['pot']
    del p['ratings'][0]['fuzz']
    del p['ratings'][0]['skills']
    del p['ratings'][0]['pos']

    for k,v in zip(rate_to_use,stats):
        p['ratings'][0][k] = int(v)

In [ ]:
with open('1984_fixed.json','wt') as fp:
    json.dump(roster,fp)